In [38]:
import ee
import geemap
import pprint
from matplotlib import pyplot as plt
import pandas as pd
import os
import numpy as np
from pprint import pprint
import copy
import string
from io import StringIO
from pandas.compat import StringIO



from sklearn.metrics import confusion_matrix, classification_report
#from sklearn.metrics import classification_report

In [39]:
# Initialise (don't need maybe?)
ee.Initialize()
# Configure the pretty printing output & initialize earthengine
#pp = pprint.PrettyPrinter(depth=4)

## Parameters (INPUT)

In [40]:
out_dir = os.path.join(os.path.expanduser('~'), 'gee/Japan_mapping/results/validation/')

## Load maps and prefecures

In [41]:
# Load rice maps
tambo1519_path = 'users/luiscartor/tambo_1519_filtered'
tambo1519 = geemap.ee.Image(tambo1519_path)



tambo1014_path = 'users/luiscartor/tambo_1014_filtered'
tambo1014 = geemap.ee.Image(tambo1014_path)
tambo0509_path = 'users/luiscartor/tambo_0509_filtered'
tambo0509 = geemap.ee.Image(tambo0509_path)
tambo0004_path = 'users/luiscartor/tambo_0004_filtered'
tambo0004 = geemap.ee.Image(tambo0004_path)
tambo9599_path = 'users/luiscartor/tambo_9599_filtered'
tambo9599 = geemap.ee.Image(tambo9599_path)
tambo9094_path = 'users/luiscartor/tambo_9094_filtered'
tambo9094 = geemap.ee.Image(tambo9094_path)
tambo8589_path = 'users/luiscartor/tambo_8589_filtered'
tambo8589 = geemap.ee.Image(tambo8589_path)

# Load JAXA map
lc2015_path = 'users/luiscartor/jaxaLC2015'
#lc2015_path = '/Users/luis/Documents/research/jsps/ricemapping/data/JaxaLandCover/jaxaLC2015.tif'
lc2015 = geemap.ee.Image(lc2015_path)

# Load prefectures and Japan shapefiles (from GEE account)
japan_shp = 'users/luiscartor/japan_gadm0'
japan = geemap.ee.FeatureCollection(japan_shp)

prefs_shp = 'users/luiscartor/japan_gadm1_noOKINAWA'
prefs = geemap.ee.FeatureCollection(prefs_shp)


# List of prefectures transplanting START DOY: We extract from here the used prefectures
transplant_startdates_table = '../Japan_mapping/data/transplanting_startdates.csv'
transplant_startdates = pd.read_csv(transplant_startdates_table, sep=",")

print(transplant_startdates)

   Prefecture  8589  9094  9599  0004  0509  1014  1519
0    Hokkaido   142   142   142   140   140   142   139
1      Aomori   134   134   133   132   135   136   135
2       Iwate   128   130   130   130   130   132   130
3      Miyagi   123   124   123   123   124   125   124
4       Akita   134   133   133   133   133   136   135
5    Yamagata   133   133   132   133   133   135   133
6   Fukushima   129   130   128   128   127   131   129
7     Ibaraki   120   121   120   118   117   119   117
8     Tochigi   124   121   121   120   120   121   122
9       Gunma   158   148   141   141   143   143   142
10    Saitama   124   123   121   119   119   118   118
11      Chiba   117   116   114   109   109   109   108
12      Tokyo   156   153   149   151   148   152   148
13   Kanagawa   148   145   143   144   142   140   139
14    Niigata   123   122   121   124   125   125   124
15     Toyama   121   120   119   122   125   124   122
16   Ishikawa   120   120   119   120   121   12

## Process layers

### Extract paddy layers

In [42]:
jaxa_rice = lc2015.eq(3)
jaxa_ricemask = jaxa_rice.updateMask(jaxa_rice)
jaxa_NOricemask = jaxa_rice.updateMask(jaxa_rice.Not())



## Visualize maps

In [52]:
Map = geemap.Map(center=[38,138], zoom=6)
Map.add_basemap('HYBRID')
Map

Map(center=[38, 138], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [44]:
#Map.addLayer(japan, {}, 'Japan')
Map.addLayer(prefs, {}, 'Prefectures')
# Define a palette for the 18 distinct land cover classes.
igbpPalette = [
  'aec3d4', # water
  '387242', # forest
  '91af40', ## shrub, grass
  '111149', ## wetlands
  'cdb33b', ## croplands
  'cc0013', ## urban
  '33280d', ## crop mosaic
  'd7cdcc', ## snow and ice
  'f7e084', ## barren
  '6f6f6f'  ## tundra
];
#Map.addLayer(lc2015,{min: 1, max: 10, 'palette': igbpPalette},'LC2015')
Map.addLayer(lc2015,{min: 0, max: 10},'LC2015')
#Map.addLayer(jaxa_rice, {},'jaxaRice')
#Map.addLayer(jaxa_ricemask, {},'jaxaRiceMask')
Map.addLayer(tambo1519,{min: 0, max: 1},'tambo1519')

# JAXA MAP VALIDATION

## Extract sample points

In [45]:
def rename(feat):
    return ee.Feature(feat.geometry(), { 
        'tambovalue': feat.get('b1')
    })

tambomap = tambo1519

#tambo_ricemask = tambomap.updateMask(tambomap)
#tambo_NOricemask = tambomap.updateMask(tambomap.Not())

# Create empty feature collection to store sampled points
samplelist = list()

# Sample rice per prefecture
# Prefecture loop 
for prefecture in transplant_startdates['Prefecture']:
    #print(prefecture)
    # Extract prefecture polygon
    prefpolygon = prefs \
            .filter(ee.Filter.eq("NAME_1", prefecture)) 
    # Stratified sample
    ricesample = tambomap.stratifiedSample(**{
        'numPoints': 50,
        'region': prefpolygon,
        'seed': 0,
        'geometries': True
    })
    
    # Rename field "b1" (causes problem later when extracting values for other layers)
    ricesample = ricesample.map(rename)
    
    # Add prefecture to sample
    def addPref(feat):
        return feat.set('Prefecture',prefecture)
    ricesample = ricesample.map(addPref)
    
    # Adds prefecture sample to list
    samplelist.append(ricesample)

In [48]:
# Merge features from list (merges all prefectures)
sample = samplelist[0]
for i in range(1,len(samplelist)):
        sample = sample.merge(samplelist[i])

# Shapefile too big so we export to drive
#filename = '../Japan_mapping/results/validation/samplepointsvalidation.shp'
#geemap.ee_export_vector_to_drive(sample, 'validationpoints.shp', folder='GEEexports')

task = ee.batch.Export.table.toAsset(**{
    'collection': sample,
    'description':'validationpoints',
    'assetId': 'users/luiscartor/validationpoints'
})
task.start()

In [49]:
sample = samplelist[0]
for i in range(1,len(samplelist)-1):
        sample = sample.merge(samplelist[i])
#print(samplelist[45].getInfo())

In [50]:
# Load validation sample from drive
sample_shp = 'users/luiscartor/validationpoints'
sample = geemap.ee.FeatureCollection(sample_shp)
#print(sample.getInfo())

In [51]:
Map.addLayer(sample, {},'sample')
Map.addLayer(tambo1519, {'min':0,'max':1}, 'tambo')
#print(sample.getInfo())

In [ ]:
## Extract Jaxa LCM values

In [53]:
# Export shp
jaxavalidation_shp = os.path.join('../Japan_mapping/results/validation/', 'jaxavalidationpoints.shp')
geemap.extract_values_to_points(sample, lc2015, jaxavalidation_shp)

# Export csv
jaxavalidation_csv = os.path.join('../Japan_mapping/results/validation/', 'jaxavalidationpoints.csv')
geemap.extract_values_to_points(sample, lc2015, jaxavalidation_csv)

Generating URL ...
Please wait ...
Data downloaded to /Users/luis/gee/Japan_mapping/results/validation/jaxavalidationpoints.shp
Generating URL ...
Please wait ...
Data downloaded to /Users/luis/gee/Japan_mapping/results/validation/jaxavalidationpoints.csv


## Confusion Matrix

In [54]:
# Load validation points table
validationpoints_table = '../Japan_mapping/results/validation/jaxavalidationpoints.csv'
validationpoints = pd.read_csv(validationpoints_table, sep=",")
validationpoints

,Prefecture,tambovalue,system:index,b1
0,Yamagata,0,00000000000000000dcb_0,4
1,Yamagata,0,00000000000000000dcc_0,4
2,Yamagata,0,00000000000000000dcd_0,6
3,Yamagata,0,00000000000000000dce_0,7
4,Yamagata,0,00000000000000000dcf_0,6
5,Yamagata,0,00000000000000000dd0_0,6
6,Yamagata,0,00000000000000000dd1_0,6
7,Yamagata,0,00000000000000000dd2_0,7
8,Yamagata,0,00000000000000000dd3_0,7
9,Yamagata,0,00000000000000000dd4_0,3


In [55]:
# Create confusion matrix all JAXA LC types
confmat = confusion_matrix(validationpoints.tambovalue,validationpoints.b1)
print(confmat)
print(classification_report(validationpoints.tambovalue,validationpoints.b1))

[[   0   41  206  139  161   95  493  164  308  661   32]
 [   1   24   44 2003  170   20   16    3    1    6   12]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2300
           1       0.37      0.01      0.02      2300
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00   

/Users/luis/opt/anaconda3/envs/gee/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/luis/opt/anaconda3/envs/gee/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/luis/opt/anaconda3/envs/gee/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
# Create confusion matrix rice VS NOrice
validationbinary = copy.deepcopy(validationpoints)
validationbinary.loc[(validationbinary.b1 != 3),'b1'] = 0
validationbinary.loc[(validationbinary.b1 == 3),'b1'] = 1

confmat_bin = confusion_matrix(validationbinary.tambovalue,validationbinary.b1)
print('Confusion matrix')
print(confmat_bin)

print('Normalized confusion matrix')
confmat_bin_norm = confmat_bin / confmat_bin.sum(axis=1)
print(confmat_bin_norm)
print(classification_report(validationbinary.tambovalue,validationbinary.b1))

Confusion matrix
[[2161  139]
 [ 297 2003]]
Normalized confusion matrix
[[0.93956522 0.06043478]
 [0.12913043 0.87086957]]
              precision    recall  f1-score   support

           0       0.88      0.94      0.91      2300
           1       0.94      0.87      0.90      2300

    accuracy                           0.91      4600
   macro avg       0.91      0.91      0.91      4600
weighted avg       0.91      0.91      0.91      4600



## Prefecture-level accuracy reports

In [57]:
precision = []
recall = []
data = []
# Loop for each prefecture
for prefecture in transplant_startdates['Prefecture']:

    validationbinary_pref = validationbinary.loc[validationbinary["Prefecture"]==prefecture]
    confmatpref = confusion_matrix(validationbinary_pref.tambovalue,validationbinary_pref.b1)
    confmatpref_norm = confmatpref / confmatpref.sum(axis=1)
    #precision.append(classification_report(validationbinary_pref.tambovalue,validationbinary_pref.b1,output_dict=True)['1']['precision'])
    #recall.append(classification_report(validationbinary_pref.tambovalue,validationbinary_pref.b1,output_dict=True)['1']['recall'])
    print(prefecture)
    print(confmatpref)
    print(confmatpref_norm)
    data.append([prefecture,
                 classification_report(validationbinary_pref.tambovalue,validationbinary_pref.b1,output_dict=True)['1']['precision'],
                classification_report(validationbinary_pref.tambovalue,validationbinary_pref.b1,output_dict=True)['1']['recall']])




Hokkaido
[[50  0]
 [ 9 41]]
[[1.   0.  ]
 [0.18 0.82]]
Aomori
[[47  3]
 [ 4 46]]
[[0.94 0.06]
 [0.08 0.92]]
Iwate
[[50  0]
 [ 9 41]]
[[1.   0.  ]
 [0.18 0.82]]
Miyagi
[[45  5]
 [ 8 42]]
[[0.9  0.1 ]
 [0.16 0.84]]
Akita
[[49  1]
 [ 0 50]]
[[0.98 0.02]
 [0.   1.  ]]
Yamagata
[[49  1]
 [ 1 49]]
[[0.98 0.02]
 [0.02 0.98]]
Fukushima
[[48  2]
 [ 4 46]]
[[0.96 0.04]
 [0.08 0.92]]
Ibaraki
[[42  8]
 [ 2 48]]
[[0.84 0.16]
 [0.04 0.96]]
Tochigi
[[42  8]
 [ 1 49]]
[[0.84 0.16]
 [0.02 0.98]]
Gunma
[[49  1]
 [ 3 47]]
[[0.98 0.02]
 [0.06 0.94]]
Saitama
[[43  7]
 [ 1 49]]
[[0.86 0.14]
 [0.02 0.98]]
Chiba
[[45  5]
 [ 1 49]]
[[0.9  0.1 ]
 [0.02 0.98]]
Tokyo
[[47  3]
 [26 24]]
[[0.94 0.06]
 [0.52 0.48]]
Kanagawa
[[47  3]
 [ 3 47]]
[[0.94 0.06]
 [0.06 0.94]]
Niigata
[[50  0]
 [ 1 49]]
[[1.   0.  ]
 [0.02 0.98]]
Toyama
[[45  5]
 [ 1 49]]
[[0.9  0.1 ]
 [0.02 0.98]]
Ishikawa
[[46  4]
 [ 1 49]]
[[0.92 0.08]
 [0.02 0.98]]
Fukui
[[47  3]
 [ 4 46]]
[[0.94 0.06]
 [0.08 0.92]]
Yamanashi
[[49  1]
 [ 3 47]]
[[0.98 0

In [58]:
prefacctable = pd.DataFrame(data, columns = ['Prefecture','precision','recall']) 
print(prefacctable)
#../Japan_mapping/results/validation/
#table_path = os.path.join(out_dir, 'prefecturesaccuracies.csv')
#prefacctable.to_csv(table_path, index = False)


   Prefecture  precision  recall
0    Hokkaido   1.000000    0.82
1      Aomori   0.938776    0.92
2       Iwate   1.000000    0.82
3      Miyagi   0.893617    0.84
4       Akita   0.980392    1.00
5    Yamagata   0.980000    0.98
6   Fukushima   0.958333    0.92
7     Ibaraki   0.857143    0.96
8     Tochigi   0.859649    0.98
9       Gunma   0.979167    0.94
10    Saitama   0.875000    0.98
11      Chiba   0.907407    0.98
12      Tokyo   0.888889    0.48
13   Kanagawa   0.940000    0.94
14    Niigata   1.000000    0.98
15     Toyama   0.907407    0.98
16   Ishikawa   0.924528    0.98
17      Fukui   0.938776    0.92
18  Yamanashi   0.979167    0.94
19     Nagano   1.000000    1.00
20       Gifu   0.976190    0.82
21   Shizuoka   0.979167    0.94
22      Aichi   0.784314    0.80
23        Mie   0.979592    0.96
24      Shiga   1.000000    0.90
25      Kyoto   0.940000    0.94
26      Osaka   0.928571    0.78
27      Hyogo   0.888889    0.96
28       Nara   1.000000    0.84
29   Wakay

# VISUAL INTERPRETATION ASSESSMENT

## Extract sample points

In [59]:
def rename(feat):
    return ee.Feature(feat.geometry(), { 
        'tambovalue': feat.get('b1')
    })

tambomap = tambo1519

# Create empty feature collection to store sampled points
visualsamplelist = list()

# Sample rice per prefecture
# Prefecture loop 
for prefecture in transplant_startdates['Prefecture']:
    #print(prefecture)
    # Extract prefecture polygon
    prefpolygon = prefs \
            .filter(ee.Filter.eq("NAME_1", prefecture)) 
    # Stratified sample
    ricesample = tambomap.stratifiedSample(**{
        'numPoints': 5,
        'region': prefpolygon,
        'seed': 0,
        'geometries': True
    })
    
    # Rename field "b1" (causes problem later when extracting values for other layers)
    ricesample = ricesample.map(rename)
    
    # Add prefecture to sample
    def addPref(feat):
        return feat.set('Prefecture',prefecture)
    ricesample = ricesample.map(addPref)
    
    # Adds prefecture sample to list
    visualsamplelist.append(ricesample)
    

# Merge features from list (merges all prefectures)
visualsample = visualsamplelist[0]
for i in range(1,len(visualsamplelist)):
        visualsample = visualsample.merge(visualsamplelist[i])

# Shapefile too big so we export to drive (we don't need export to asset)
#geemap.ee_export_vector_to_drive(visualsample, 'visualvalidationpoints.shp', folder='GEEexports')

task = ee.batch.Export.table.toAsset(**{
    'collection': visualsample,
    'description':'visualvalidationpoints',
    'assetId': 'users/luiscartor/visualvalidationpoints'
})
task.start()

In [61]:
# Load validation sample from drive
visualsample_shp = 'users/luiscartor/visualvalidationpoints'
visualsample = geemap.ee.FeatureCollection(visualsample_shp)
#print(visualsample.getInfo())

### Visualize points

In [62]:
Map2 = geemap.Map(center=[38,138], zoom=6)
Map2.add_basemap('HYBRID')
Map2

Map(center=[38, 138], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [63]:
Map2.addLayer(visualsample, {},'sample')

## Extract tambo values for other periods

In [65]:
periods_dict = {'8589': tambo8589,
              '9094': tambo9094,
              '9599': tambo9599,
              '0004': tambo0004,
              '0509': tambo0509,
              '1014': tambo1014,
               '1519': tambo1519}

for period in periods_dict.values():
    
    # Export shp
    shpfilename = list(periods_dict.keys())[list(periods_dict.values()).index(period)] + '_visualvalidationpoints.shp'
    visualvalidation_shp = os.path.join('../Japan_mapping/results/validation/', shpfilename)
    geemap.extract_values_to_points(visualsample, period, visualvalidation_shp)

    #csvfilename = list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)] + '_visualvalidationpoints.csv'
    #visualvalidation_csv = os.path.join('../Japan_mapping/results/validation/', csvfilename)
    #geemap.extract_values_to_points(visualsample, period, visualvalidation_csv)

Generating URL ...
Please wait ...
Data downloaded to /Users/luis/gee/Japan_mapping/results/validation/8589_visualvalidationpoints.shp
Generating URL ...
Please wait ...
Data downloaded to /Users/luis/gee/Japan_mapping/results/validation/9094_visualvalidationpoints.shp
Generating URL ...
Please wait ...
Data downloaded to /Users/luis/gee/Japan_mapping/results/validation/9599_visualvalidationpoints.shp
Generating URL ...
Please wait ...
Data downloaded to /Users/luis/gee/Japan_mapping/results/validation/0004_visualvalidationpoints.shp
Generating URL ...
Please wait ...
Data downloaded to /Users/luis/gee/Japan_mapping/results/validation/0509_visualvalidationpoints.shp
Generating URL ...
Please wait ...
Data downloaded to /Users/luis/gee/Japan_mapping/results/validation/1014_visualvalidationpoints.shp
Generating URL ...
Please wait ...
Data downloaded to /Users/luis/gee/Japan_mapping/results/validation/1519_visualvalidationpoints.shp


## Visual interpretation (non-scripted)

Exported shapefiles are merged in QGIS and each column renamed to each period. One empty column for each period is created in order to add rice balues based on visual interpretation of aerial photography imagery.

## Import visual interpreted values tables